# Import Dependencies

In [2]:
from transformers import pipeline
import os
import xml.etree.ElementTree as ET

D:\Apps\Anaconda\envs\ds\lib\site-packages\torchaudio\backend\utils.py:67: UserWarning: No audio backend is available.
  warnings.warn('No audio backend is available.')


## Classes

In [3]:
class Opinion:
    def __init__(self, target, category, polarity, start, end):
        self.target = target
        self.category = category
        self.polarity = polarity
        self.start = start
        self.end = end

In [4]:
class Sentence:
    def __init__(self, text, opinions):
        self.text = text
        self.opinions = opinions

In [5]:
class Review:
    def __init__(self, sentences):
        self.sentences = sentences

# Instantiate Model

In [6]:
sentiment_analyzer = pipeline("sentiment-analysis")

Downloading: 100%|██████████| 629/629 [00:00<00:00, 314kB/s]
Downloading: 100%|██████████| 268M/268M [00:04<00:00, 66.4MB/s]
Downloading: 100%|██████████| 232k/232k [00:00<00:00, 471kB/s]
Downloading: 100%|██████████| 48.0/48.0 [00:00<00:00, 24.0kB/s]


# Encode and Calculate Sentiments

In [9]:
xml_data = open(os.path.join('data/semeval16_restaurants_train.xml'), 'r').read()
root = ET.XML(xml_data)

reviews = []

for _, review in enumerate(root):
    current_sentences = []

    for _, sentences in enumerate(review):
        for _, sentence in enumerate(sentences):
            opinions = []
            text = ""

            for _, sentence_children in enumerate(sentence):
                if sentence_children.tag == 'Opinions':
                    for _, opinion in enumerate(sentence_children):
                        op = Opinion(
                            target = opinion.attrib["target"] if "target" in opinion.attrib else "",
                            category = opinion.attrib["category"] if "category" in opinion.attrib else "",
                            polarity = opinion.attrib["polarity"] if "polarity" in opinion.attrib else "",
                            start = opinion.attrib["from"] if "from" in opinion.attrib else "",
                            end = opinion.attrib["to"] if "to" in opinion.attrib else ""
                        )
                        opinions.append(op)
                if sentence_children.tag == "text":
                    text = sentence_children.text
                    # text = ''

            sen = Sentence(text, opinions)
            current_sentences.append(sen)
            
    newReview = Review(current_sentences)
    reviews.append(newReview)

In [21]:
correct_identified = 0
wrong_identified = 0
number_of_neutrals = 0

for review in reviews:
    for sentence in review.sentences:
        result = sentiment_analyzer(sentence.text)[0]
        if (len(sentence.opinions) > 0):
            original_polarity = sentence.opinions[0].polarity

            if (original_polarity == 'neutral'):
                number_of_neutrals = number_of_neutrals + 1

            if (original_polarity.upper() == result['label']):
                correct_identified = correct_identified + 1
            else:
                wrong_identified = wrong_identified + 1
                print(f"{sentence.text} ---- Expected {original_polarity.upper()} --- Identified {result['label']}")

print(f"Correct identified: {correct_identified}")
print(f"Wrong identified: {wrong_identified})
print(f"Number of neutrals: {number_of_neutrals}")

For the price, you cannot eat this well in Manhattan. ---- Expected POSITIVE --- Identified NEGATIVE
Food was okay, nothing great. ---- Expected NEUTRAL --- Identified POSITIVE
Can't wait wait for my next visit. ---- Expected POSITIVE --- Identified NEGATIVE
The only thing I moderately enjoyed was their Grilled Chicken special with Edamame Puree. ---- Expected POSITIVE --- Identified NEGATIVE
Their sake list was extensive, but we were looking for Purple Haze, which wasn't listed but made for us upon request! ---- Expected POSITIVE --- Identified NEGATIVE
the food is decent. ---- Expected NEUTRAL --- Identified POSITIVE
i know, you were too busy showing off your vintage tee shirt and looking bored, but my agenda is i'm here to eat and enjoy the company of friends, seeking a pleasant experience. ---- Expected NEGATIVE --- Identified POSITIVE
well, i didn't find it there, and trust, i have told everyone i can think of about my experience.  ---- Expected NEGATIVE --- Identified POSITIVE
I 

Correct identified: 1419
Wrong identified: 289
Number of neutrals: 78